# Advent of Code 2022
## Day 22
*<https://adventofcode.com/2022/day/22>*

In [1]:
import heapq
import math
import re
from collections import Counter, defaultdict, deque, namedtuple
from itertools import combinations, permutations, product
from string import ascii_letters, ascii_lowercase, ascii_uppercase

import IPython
import z3
from rich import inspect, pretty, print

from new_helper import *

pretty.install()

In [2]:
DAY = 22
inp = get_aoc_input(DAY, 2022).parse_groups()
part_1 = part_2 = 0

In [3]:
m = Grid(inp[0], default=" ")
instructions = []

i = 0
while i < len(inp[1][0]):
    match inp[1][0][i]:
        case "L":
            instructions.append("L")
            i += 1
        case "R":
            instructions.append("R")
            i += 1
        case _:
            n = ints(inp[1][0][i:])[0]
            instructions.append(n)
            i += len(str(n))

In [4]:
def first_non_empty(l: list[str]) -> int:
    for i, x in enumerate(l):
        if x != " ":
            return i
    raise ValueError("No non-empty found")


def move(m: Grid[str], x, y, dx, dy):
    nx, ny = x + dx, y + dy
    if not m.in_bounds(nx, ny) or m[nx, ny] == " ":
        match dx, dy:
            case 1, 0:
                nx = first_non_empty(m[:, y])
            case -1, 0:
                nx = m.width - first_non_empty(m[::-1, y]) - 1
            case 0, 1:
                ny = first_non_empty(m[x, :])
            case 0, -1:
                ny = m.height - first_non_empty(m[x, ::-1]) - 1

    match m[nx, ny]:
        case "#":
            return x, y
        case ".":
            return nx, ny
        case _:
            raise ValueError("Unknown: ", m[nx, ny], nx, ny)

In [5]:
def rotate_left(dx: int, dy: int) -> tuple[int, int]:
    return dy, -dx

def rotate_right(dx: int, dy: int) -> tuple[int, int]:
    return -dy, dx

In [6]:
x, y = m.row(0).index("."), 0
dx, dy = 1, 0

for ins in instructions:
    match ins:
        case "L":
            dx, dy = rotate_left(dx, dy)
        case "R":
            dx, dy = rotate_right(dx, dy)
        case n:
            assert isinstance(n, int)
            for _ in range(n):
                x, y = move(m, x, y, dx, dy)

In [7]:
def score(x, y, dx, dy):
    n = 1000 * (y + 1) + 4 * (x + 1)
    match dx, dy:
        case 1, 0:
            n += 0
        case 0, 1:
            n += 1
        case -1, 0:
            n += 2
        case 0, -1:
            n += 3
    return n

part_1 = score(x, y, dx, dy)

In [8]:
TELEPORTS = {}
S = int(math.sqrt((m.count("#") + m.count(".")) // 6))


def add_teleport(x: int, y: int, dx: int, dy: int, nx: int, ny: int, ndx: int, ndy: int):
    TELEPORTS[x, y, dx, dy] = nx, ny, ndx, ndy
    TELEPORTS[nx, ny, -ndx, -ndy] = x, y, -dx, -dy


# Hardcoded for advent of code input. Does not work with sample.
for i in range(S):
    add_teleport(S, i, -1, 0, 0, S * 3 - i - 1, 1, 0)
    add_teleport(S + i, 0, 0, -1, 0, S * 3 + i, 1, 0)
    add_teleport(2 * S + i, 0, 0, -1, i, 4 * S - 1, 0, -1)
    add_teleport(3 * S - 1, i, 1, 0, 2 * S - 1, 3 * S - i - 1, -1, 0)
    add_teleport(2 * S + i, S - 1, 0, 1, 2 * S - 1, S + i, -1, 0)
    add_teleport(S + i, 3 * S - 1, 0, 1, S - 1, 3 * S + i, -1, 0)
    add_teleport(S, S + i, -1, 0, i, 2 * S, 0, 1)


def move_2(m: Grid[str], x, y, dx, dy) -> tuple[int, int, int, int]:
    nx, ny = x + dx, y + dy
    ndx, ndy = dx, dy
    if not m.in_bounds(nx, ny) or m[nx, ny] == " ":
        nx, ny, ndx, ndy = TELEPORTS[x, y, dx, dy]

    match m[nx, ny]:
        case "#":
            return x, y, dx, dy
        case ".":
            return nx, ny, ndx, ndy
        case _:
            raise ValueError(f"Unknown value at ({nx}, {ny}): '{m[nx, ny]}'.")

In [9]:
x, y = m.row(0).index("."), 0
dx, dy = 1, 0

for ins in instructions:
    match ins:
        case "L":
            dx, dy = rotate_left(dx, dy)
        case "R":
            dx, dy = rotate_right(dx, dy)
        case n:
            assert isinstance(n, int)
            for _ in range(n):
                x, y, dx, dy = move_2(m, x, y, dx, dy)

part_2 = score(x, y, dx, dy)

In [10]:
print_part_1(part_1)
print_part_2(part_2)

Part 1: 93226

Part 2: 37415

In [11]:
# submit_part_1(part_1, DAY, 2022)
# submit_part_2(part_2, DAY, 2022)